In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.results_extraction import extract_results, \
                                       results_by_subdistribution, \
                                       results_by_named_entity, \
                                       confusion_matrix
from modules.tools import find

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

from estnltk.taggers import VabamorfCorpusTagger
vm_corpus_tagger = VabamorfCorpusTagger()

### Flags & variables:

In [2]:
use_vabamorfcorpustagger = False

In [3]:
divided_corpus = os.path.join('..', 'data', 'divided_corpus.txt')
json_files_location = os.path.join('..', 'data', 'vallakohtufailid-json-flattened')
vallakohtufailid_location = os.path.join('..', 'data', 'vallakohtufailid')
no_goldstandard_tags_location = os.path.join('..', 'data', 'files_without_goldstandard_annotations.txt')
testing_files_location = os.path.join('..', 'data', 'vallakohtufailid-json-flattened')

removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

---

Get files without goldstandard annotations

In [4]:
with open(no_goldstandard_tags_location, 'r', encoding='UTF-8') as in_f:
    lines = in_f.readlines()

no_goldstandard_annotations = [line.strip() for line in lines]

Get all files for the first five subdistributions

In [5]:
files = {}

with open(divided_corpus, 'r', encoding = 'UTF-8') as in_f:
    txt = in_f.readlines()

for filename in txt:
    file, subdistribution = filename.split(":")
    files[file] = subdistribution.strip()

Return the subdistribution for testing and training
(e.g. `1` to test, `2`;`3`;`4`;`5` to train or `2` to test, `1` and `3`;`4`;`5` to train)

In [6]:
def get_testing_and_training_subdistribution(subdistribution):
    training = []
    for y in sorted(set(files.values())):
        if int(y) == subdistribution:
            testing = int(y)
        else:
            training.append(int(y))
    return testing, training

Create a list of Text objects from the files read in before (the subdistributions meant for training)

In [7]:
def create_training_texts(filenames):
    print('(!) Preparing training texts')
    
    training_texts = []
    
    for filename in filenames:
        if filename in no_goldstandard_annotations:
            continue
        else:
            with open(os.path.join(json_files_location, filename), 'r', encoding='UTF-8') as in_f:
                tagged_text = preprocess_text(json_to_text(in_f.read()))
                
            if use_vabamorfcorpustagger:
                tagged_text.pop_layer('morph_analysis')
                vm_corpus_tagger.tag([tagged_text])

            training_texts.append(tagged_text)
            
    print('(!) Training texts done')
    return training_texts

Train the NerTagger model using settings from the model directory

In [8]:
def train_nertagger(training_texts, new_model_dir):
    print('(!) Training NerTagger')
    
    modelUtil = ModelStorageUtil(new_model_dir)
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
    print('(!) NerTagger training done\n')

Tag the files by finding the appropriate from vallakohtufailid_location,
then preprocessing them, removing layers for optimal file sizes and
saving them to a new directory in the model folder.

In [9]:
def tag_files(model_dir, testing_files, use_vabamorfcorpustagger, tag_wordner):
    nertagger = NerTagger(model_dir)
    
    print("(!) Tagging...")
    iterator = 1
    for test_file in testing_files:
        with open(find(test_file.replace(".json", ".txt"), vallakohtufailid_location), 'r', encoding='UTF-8') as f:
            text = f.read()
        
        if test_file == "Tartu_V6nnu_Ahja_id3502_1882a.json":
            text = text.replace('..', '. .')
        text = preprocess_text(Text(text))

        if use_vabamorfcorpustagger or "vabamorf" in model_dir:
            text.pop_layer('morph_analysis')
            text = [text]
            vm_corpus_tagger.tag( text )
            text = text[0]
        nertagger.tag(text)
        text.add_layer(flatten(text['ner'], 'flat_ner'))
        
        if tag_wordner:
            print('(!) Tagging Word Level NER')
            wordnertagger = WordLevelNerTagger(model_dir)
            wordnertagger.tag(text)
            text.add_layer(flatten(text['wordner'], 'flat_wordner'))
        
        for x in removed_layers:
            text.pop_layer(x)

        path = os.path.join(model_dir, 'vallakohtufailid-trained-nertagger')
        if not os.path.exists(path):
            os.mkdir(path)
            
        text_to_json(text, file=os.path.join(model_dir, 'vallakohtufailid-trained-nertagger', test_file))
        
        print(f'{iterator}. Tagged file {test_file}')
        iterator += 1
    print('(!) Files tagged')


In [10]:
# Train the model by applying all necessary 
def train_model(model_directory, tag_wordner):
    for subdistribution in sorted(set(files.values())):
        testing, training = get_testing_and_training_subdistribution(int(subdistribution))

        # Get the filenames to be trained on from the files dictionary
        filenames = [key for key, value in files.items() if int(value) in training]

        # Create training_texts from the aforementioned filenames
        training_texts = create_training_texts(filenames)

        # Set up the trainer and training
        new_model_dir = os.path.join('models', model_directory)
        train_nertagger(training_texts, new_model_dir)

        # Set up the new trained nertagger and defining layers to be removed later on
        tagger = NerTagger(model_dir = new_model_dir)
        #print(tagger.nersettings)
        # Tag the files using the new nertagger
        testing_files = [key for key, value in files.items() if int(value) == testing]
        tag_files(new_model_dir, testing_files, use_vabamorfcorpustagger, tag_wordner)
            
    # Get results of model
    extract_results(files,
                    no_goldstandard_annotations,
                    os.path.join('models', model_directory, 'vallakohtufailid-trained-nertagger'), #training files location
                    testing_files_location,
                    os.path.join('models', model_directory)) #results.txt location
    
    print(f"(!) Model {model_directory} trained")

NB! Make sure the values (location of the training files and location of the `results.txt` file) in the `extract_results()` function are correct as these cannot be referenced before the model directory is defined.

To train the model the `model_directory` (given to the `train_model()` function) must contain a `settings.py` file

In [ ]:
models = [
    'model_default_with_vabamorftagger',
    'model_local_features_without_morph',
    'model_morph_without_lemmas',
    'model_morph_with_lemmas',
    'model_morph_with_lemmas_and_sentences',
    'model_morph_with_lemmas_and_sentences_and_gazzetteer',
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_initial'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer2'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer1and2'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc_variants')    
]

for model in models:
    train_model(model, tag_wordner = False)

In [11]:
path = os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc_variants')

train_model(path, tag_wordner = True)

(!) Preparing training texts
(!) Training texts done
(!) Training NerTagger
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 347546
Seconds required: 2.086

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30474.851022
Trial #1 (eta = 0.100000): 2509.798853
Trial #2 (eta = 0.200000): 3571.168525
Trial #3 (eta = 0.400000): 6982.558767
Trial #4 (eta = 0.800000): 13567.187870
Trial #5 (eta = 1.600000): 33517.948069 (worse)
Trial #6 (eta = 0.050000): 2188.020241
Trial #7 (eta = 0.025000): 2373.892858
Trial #8 (eta = 

***** Epoch #37 *****
Loss: 451.173951
Improvement ratio: 0.306801
Feature L2-norm: 104.994102
Learning rate (eta): 0.049816
Total number of feature updates: 600991
Seconds required for this iteration: 0.336

***** Epoch #38 *****
Loss: 436.505935
Improvement ratio: 0.286233
Feature L2-norm: 105.614194
Learning rate (eta): 0.049811
Total number of feature updates: 617234
Seconds required for this iteration: 0.326

***** Epoch #39 *****
Loss: 427.891405
Improvement ratio: 0.290678
Feature L2-norm: 106.222061
Learning rate (eta): 0.049806
Total number of feature updates: 633477
Seconds required for this iteration: 0.340

***** Epoch #40 *****
Loss: 413.717372
Improvement ratio: 0.322163
Feature L2-norm: 106.824378
Learning rate (eta): 0.049801
Total number of feature updates: 649720
Seconds required for this iteration: 0.346

***** Epoch #41 *****
Loss: 416.581246
Improvement ratio: 0.207907
Feature L2-norm: 107.410644
Learning rate (eta): 0.049796
Total number of feature updates: 665963

***** Epoch #79 *****
Loss: 248.877650
Improvement ratio: 0.118087
Feature L2-norm: 122.925089
Learning rate (eta): 0.049608
Total number of feature updates: 1283197
Seconds required for this iteration: 0.317

***** Epoch #80 *****
Loss: 251.542934
Improvement ratio: 0.084226
Feature L2-norm: 123.221516
Learning rate (eta): 0.049603
Total number of feature updates: 1299440
Seconds required for this iteration: 0.315

***** Epoch #81 *****
Loss: 235.989812
Improvement ratio: 0.067638
Feature L2-norm: 123.517995
Learning rate (eta): 0.049598
Total number of feature updates: 1315683
Seconds required for this iteration: 0.358

***** Epoch #82 *****
Loss: 244.563722
Improvement ratio: 0.177566
Feature L2-norm: 123.812238
Learning rate (eta): 0.049593
Total number of feature updates: 1331926
Seconds required for this iteration: 0.645

***** Epoch #83 *****
Loss: 243.362781
Improvement ratio: 0.098897
Feature L2-norm: 124.102649
Learning rate (eta): 0.049588
Total number of feature updates: 13

(!) Tagging Word Level NER
27. Tagged file L22ne_Emmaste_Emmaste_id15087_1895a.json
(!) Tagging Word Level NER
28. Tagged file V6ru_Vastseliina_Misso_id19866_1882a.json
(!) Tagging Word Level NER
29. Tagged file J2rva_Tyri_Kirna_id24452_1880a.json
(!) Tagging Word Level NER
30. Tagged file Tartu_V6nnu_Ahja_id12638_1875a.json
(!) Tagging Word Level NER
31. Tagged file Tartu_Kodavere_Pala_id16229_1849a.json
(!) Tagging Word Level NER
32. Tagged file Harju_Rapla_Rapla_id365_1873a.json
(!) Tagging Word Level NER
33. Tagged file Viljandi_P6ltsamaa_Adavere_id20278_1890a.json
(!) Tagging Word Level NER
34. Tagged file Viljandi_Viljandi_Karula_id19366_1868a.json
(!) Tagging Word Level NER
35. Tagged file Tartu_Kodavere_Pala_id22898_1872a.json
(!) Tagging Word Level NER
36. Tagged file P2rnu_Audru_V6lla_id5904_1878a.json
(!) Tagging Word Level NER
37. Tagged file Harju_J6el2htme_J6el2htme_id7612_1869a.json
(!) Tagging Word Level NER
38. Tagged file J2rva_Peetri_V2ike-Kareda_id19169_1869a.json
(

126. Tagged file Tartu_V6nnu_Ahja_id18630_1886a.json
(!) Tagging Word Level NER
127. Tagged file V6ru_Vastseliina_Misso_id13714_1886a.json
(!) Tagging Word Level NER
128. Tagged file V6ru_R2pina_Kahkva_id24938_1868a.json
(!) Tagging Word Level NER
129. Tagged file J2rva_Tyri_S2revere_id12762_1879a.json
(!) Tagging Word Level NER
130. Tagged file Tartu_Kodavere_Alatskivi_id21934_1881a.json
(!) Tagging Word Level NER
131. Tagged file V6ru_Vastseliina_Misso_id7468_1885a.json
(!) Tagging Word Level NER
132. Tagged file Saare_K2rla_K2rla_id5736_1827a.json
(!) Tagging Word Level NER
133. Tagged file Tartu_Sangaste_Kuigatsi_id16414_1872a.json
(!) Tagging Word Level NER
134. Tagged file Harju_Kose_Kose-Uuem6isa_id2174_1867a.json
(!) Tagging Word Level NER
135. Tagged file Viljandi_P6ltsamaa_Pajusi_id2717_1871a.json
(!) Tagging Word Level NER
136. Tagged file Viljandi_P6ltsamaa_Vana-P6ltsamaa_id8104_1888a.json
(!) Tagging Word Level NER
137. Tagged file Viljandi_K6pu_Suure-K6pu_id7189_1884a.jso

...

***** Epoch #16 *****
Loss: 1002.038645
Improvement ratio: 1.981774
Feature L2-norm: 84.986986
Learning rate (eta): 0.049920
Total number of feature updates: 249824
Seconds required for this iteration: 0.326

***** Epoch #17 *****
Loss: 939.802186
Improvement ratio: 1.690263
Feature L2-norm: 86.393819
Learning rate (eta): 0.049915
Total number of feature updates: 265438
Seconds required for this iteration: 0.322

***** Epoch #18 *****
Loss: 890.885643
Improvement ratio: 1.415766
Feature L2-norm: 87.722122
Learning rate (eta): 0.049910
Total number of feature updates: 281052
Seconds required for this iteration: 0.336

***** Epoch #19 *****
Loss: 835.923823
Improvement ratio: 1.285679
Feature L2-norm: 88.981328
Learning rate (eta): 0.049905
Total number of feature updates: 296666
Seconds required for this iteration: 0.325

***** Epoch #20 *****
Loss: 787.997047
Improvement ratio: 1.128133
Feature L2-norm: 90.182740
Learning rate (eta): 0.049900
Total number of feature updates: 312280
Sec

***** Epoch #56 *****
Loss: 321.903113
Improvement ratio: 0.206285
Feature L2-norm: 114.282871
Learning rate (eta): 0.049722
Total number of feature updates: 874384
Seconds required for this iteration: 0.323

***** Epoch #57 *****
Loss: 324.435576
Improvement ratio: 0.141148
Feature L2-norm: 114.692085
Learning rate (eta): 0.049717
Total number of feature updates: 889998
Seconds required for this iteration: 0.317

***** Epoch #58 *****
Loss: 298.642348
Improvement ratio: 0.179408
Feature L2-norm: 115.098436
Learning rate (eta): 0.049712
Total number of feature updates: 905612
Seconds required for this iteration: 0.328

***** Epoch #59 *****
Loss: 316.739260
Improvement ratio: 0.171727
Feature L2-norm: 115.503341
Learning rate (eta): 0.049707
Total number of feature updates: 921226
Seconds required for this iteration: 0.311

***** Epoch #60 *****
Loss: 292.676873
Improvement ratio: 0.157564
Feature L2-norm: 115.896642
Learning rate (eta): 0.049702
Total number of feature updates: 936840

***** Epoch #98 *****
Loss: 227.076789
Improvement ratio: 0.063516
Feature L2-norm: 127.406215
Learning rate (eta): 0.049515
Total number of feature updates: 1530172
Seconds required for this iteration: 0.305

***** Epoch #99 *****
Loss: 208.020045
Improvement ratio: 0.152963
Feature L2-norm: 127.642771
Learning rate (eta): 0.049510
Total number of feature updates: 1545786
Seconds required for this iteration: 0.304

***** Epoch #100 *****
Loss: 223.668018
Improvement ratio: 0.048575
Feature L2-norm: 127.882465
Learning rate (eta): 0.049505
Total number of feature updates: 1561400
Seconds required for this iteration: 0.305

***** Epoch #101 *****
Loss: 222.577811
Improvement ratio: 0.041595
Feature L2-norm: 128.113796
Learning rate (eta): 0.049500
Total number of feature updates: 1577014
Seconds required for this iteration: 0.304

***** Epoch #102 *****
Loss: 220.763008
Improvement ratio: 0.063052
Feature L2-norm: 128.343379
Learning rate (eta): 0.049495
Total number of feature updates:

(!) Tagging Word Level NER
65. Tagged file Tartu_Kodavere_Ranna_id14234_1857a.json
(!) Tagging Word Level NER
66. Tagged file V6ru_P6lva_K2hri_id21791_1851a.json
(!) Tagging Word Level NER
67. Tagged file Tartu_V6nnu_Ahja_id23744_1896a.json
(!) Tagging Word Level NER
68. Tagged file V6ru_R2pina_Kahkva_id21602_1867a.json
(!) Tagging Word Level NER
69. Tagged file Tartu_Kodavere_Ranna_id19767_1865a.json
(!) Tagging Word Level NER
70. Tagged file Tartu_Torma_Avinurme_id23582_1872a.json
(!) Tagging Word Level NER
71. Tagged file Tartu_R6ngu_Aakre_id2550_1888a.json
(!) Tagging Word Level NER
72. Tagged file J2rva_Tyri_Kirna_id24064_1879a.json
(!) Tagging Word Level NER
73. Tagged file Tartu_V6nnu_Ahja_id19682_1888a.json
(!) Tagging Word Level NER
74. Tagged file Viljandi_Paistu_Holstre_id10573_1900a.json
(!) Tagging Word Level NER
75. Tagged file Tartu_Kodavere_Ranna_id355_1883a.json
(!) Tagging Word Level NER
76. Tagged file Harju_Hageri_Kohila_id4930_1887a.json
(!) Tagging Word Level NER


164. Tagged file P2rnu_P2rnu-Elisabethi_Sauga_id18745_1877a.json
(!) Tagging Word Level NER
165. Tagged file Tartu_V6nnu_Ahja_id18234_1886a.json
(!) Tagging Word Level NER
166. Tagged file Tartu_Torma_Avinurme_id20454_1871a.json
(!) Tagging Word Level NER
167. Tagged file Harju_Rapla_Rapla_id18671_1869a.json
(!) Tagging Word Level NER
168. Tagged file Tartu_V6nnu_Ahja_id18214_1886a.json
(!) Tagging Word Level NER
169. Tagged file Tartu_Laiuse_Kivij2rve_id1436_1856a.json
(!) Tagging Word Level NER
170. Tagged file Tartu_V6nnu_Ahja_id22561_1878a.json
(!) Tagging Word Level NER
171. Tagged file Tartu_Kodavere_Ranna_id14286_1858a.json
(!) Tagging Word Level NER
172. Tagged file P2rnu_Halliste_Penuja_id758_1885a.json
(!) Tagging Word Level NER
173. Tagged file Saare_Kihelkonna_Atla_id6893_1872a.json
(!) Tagging Word Level NER
174. Tagged file Harju_Hageri_Kohila_id10769_1873a.json
(!) Tagging Word Level NER
175. Tagged file Tartu_V6nnu_Ahja_id16318_1883a.json
(!) Tagging Word Level NER
176.